# Deploy Strands Agent to Bedrock AgentCore Runtime with AgentCore Gateway Integration

This notebook demonstrates how to create a Strands agent with UML diagram generation capabilities using Bedrock AgentCore Gateway integration.

## Architecture Overview

## What We'll Build

1. **Lambda Function**: Generates PNG UML diagrams from OpenAPI specs
2. **AgentCore Gateway**: Provides secure access to Lambda via MCP protocol
3. **Strands Agent running on Bedrock AgentCore**: Integrates with Gateway for UML generation
4. **End-to-End Testing**: Verify complete functionality

Before running this notebook, ensure your **SageMaker Execution Role** has the following AWS managed policies attached:

### Required AWS Managed Policies:

- **AmazonBedrockFullAccess**: Access to Bedrock models and AgentCore services
- **Amazon Bedrock AgentCore**: Access to Bedrock models and AgentCore services
- **AWSLambda_FullAccess**: Create, update, and invoke Lambda functions
- **AWSCodeBuildDeveloperAccess**: Build and deploy containers via CodeBuild
- **AmazonS3FullAccess**: Store UML diagrams and deployment artifacts
- **IAMFullAccess**: Create roles and policies for Lambda and Gateway
- **AmazonCognitoPowerUser**: Create OAuth authorization servers

**⚡ Important**: Without these permissions, the notebook will fail at various steps. Ensure all policies are attached before proceeding.


In [ ]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook") 

## Load the parameters

In [ ]:
print("load the data parameters....\n")
# bucket and parameter stored from Initial setup lab01
%store -r bucket
%store -r prefix
%store -r data_dir
%store -r yml_dir
%store -r uml_dir

## check all 5 values are printed and do not fail
print(bucket)
print(prefix)
print(yml_dir)
print(uml_dir)
print(data_dir)

print("\nload the vector db parameters....\n")

# vector parameters stored from Initial setup lab02
%store -r vector_host
%store -r vector_collection_arn
%store -r vector_collection_id
%store -r bedrock_kb_execution_role_arn

print(vector_host)
print(vector_collection_arn)
print(vector_collection_id)
print(bedrock_kb_execution_role_arn)

print("\nload lambda parameters....\n")

%store -r lambda_arn
%store -r lambda_function_name

print(lambda_arn)
print(lambda_function_name)

print("\nload knowledge base parameters....\n")

# Load KB ID from the original notebook if it exists
%store -r kb_id
print(f"Knowledge Base ID: {kb_id}")

# Export as environment variable
import os
os.environ['BEDROCK_KB_ID'] = kb_id

## Step 1: Install Required Dependencies

In [ ]:
# Install required packages
!pip install strands-agents bedrock-agentcore-starter-toolkit boto3 requests

## Step 2: Create Lambda Function for UML Generation

First, we'll create a Lambda function that:
- Takes OpenAPI YAML as input
- Generates PlantUML code
- Converts PlantUML to PNG using external service
- Uploads PNG to S3
- Returns S3 URI and PlantUML code

In [ ]:
# Load Lambda function code from file
with open("code/lambda_function.py", "r") as f:
    lambda_function_code = f.read()

print("✅ Lambda function code loaded from code/lambda_function.py")
print(f"Code length: {len(lambda_function_code)} characters")

## Step 3: Deploy Lambda Function

Create and deploy the Lambda function with proper IAM permissions.

### 🔧 Configuration

**⚠️ Important**: Before running the next cell, customize the bucket name:

- Replace  in  with your initials
- Example:  for John Doe
- This ensures each user gets a unique bucket name
- Avoids conflicts when multiple users run this notebook


In [ ]:
import boto3
import zipfile
import json
import time
import uuid

# Initialize AWS clients
lambda_client = boto3.client("lambda")
iam_client = boto3.client("iam")
s3_client = boto3.client("s3")

# Configuration - Customize these values
# Replace "abc" with your initials (e.g., "jdoe" for John Doe)
LAMBDA_FUNCTION_NAME = "swagger-uml-generator-abc"  # Change "abc" to your initials
BUCKET_NAME = "swagger-diagrams-bucket-abc"  # Change "abc" to your initials
GATEWAY_NAME = "uml-generator-gateway-abc"  # Change "abc" to your initials
REGION = boto3.Session().region_name

print("🚀 Starting Lambda deployment process...")
print(f"📦 Lambda Function: {LAMBDA_FUNCTION_NAME}")
print(f"🪣 S3 Bucket: {BUCKET_NAME}")
print(f"🌐 Gateway Name: {GATEWAY_NAME}")
print(f"🌍 Region: {REGION}")
print(f"💡 Note: Make sure to replace 'abc' with your initials in all names!")

In [ ]:
# Create S3 bucket for storing UML diagrams
try:
    s3_client.create_bucket(Bucket=BUCKET_NAME)
    print(f"✅ Created S3 bucket: {BUCKET_NAME}")
except s3_client.exceptions.BucketAlreadyExists:
    print(f"✅ S3 bucket already exists: {BUCKET_NAME}")
except Exception as e:
    print(f"❌ Error creating S3 bucket: {e}")

In [ ]:
# Create IAM role for Lambda
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

role_name = 'UMLLambdaExecutionRole'

try:
    role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Execution role for UML Lambda function'
    )
    role_arn = role_response['Role']['Arn']
    print(f"✅ Created IAM role: {role_arn}")
except iam_client.exceptions.EntityAlreadyExistsException:
    role_response = iam_client.get_role(RoleName=role_name)
    role_arn = role_response['Role']['Arn']
    print(f"✅ Using existing IAM role: {role_arn}")

# Attach basic Lambda execution policy
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

# Create and attach S3 access policy
s3_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:PutObjectAcl",
                "s3:GetObject"
            ],
            "Resource": f"arn:aws:s3:::{BUCKET_NAME}/*"
        }
    ]
}

try:
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName='S3AccessPolicy',
        PolicyDocument=json.dumps(s3_policy)
    )
    print("✅ Attached S3 access policy")
except Exception as e:
    print(f"❌ Error attaching S3 policy: {e}")

# Wait for IAM propagation
print("⏳ Waiting for IAM propagation...")
time.sleep(10)

In [ ]:
# Create deployment package
with zipfile.ZipFile("lambda-deployment.zip", "w") as z:
    z.write("code/lambda_function.py", "lambda_function.py")

# Deploy Lambda function with environment variables
try:
    with open("lambda-deployment.zip", "rb") as f:
        lambda_response = lambda_client.create_function(
            FunctionName=LAMBDA_FUNCTION_NAME,
            Runtime="python3.12",
            Role=role_arn,
            Handler="lambda_function.lambda_handler",
            Code={"ZipFile": f.read()},
            Timeout=30,
            MemorySize=128,
            Environment={
                "Variables": {
                    "BUCKET_NAME": BUCKET_NAME
                }
            }
        )
    print(f"✅ Created Lambda function: {lambda_response["FunctionArn"]}")
    lambda_arn = lambda_response["FunctionArn"]
except lambda_client.exceptions.ResourceConflictException:
    # Update existing function
    with open("lambda-deployment.zip", "rb") as f:
        lambda_client.update_function_code(
            FunctionName=LAMBDA_FUNCTION_NAME,
            ZipFile=f.read()
        )
    # Update environment variables
    lambda_client.update_function_configuration(
        FunctionName=LAMBDA_FUNCTION_NAME,
        Environment={
            "Variables": {
                "BUCKET_NAME": BUCKET_NAME
            }
        }
    )
    lambda_response = lambda_client.get_function(FunctionName=LAMBDA_FUNCTION_NAME)
    lambda_arn = lambda_response["Configuration"]["FunctionArn"]
    print(f"✅ Updated existing Lambda function: {lambda_arn}")

print(f"🎯 Lambda Function ARN: {lambda_arn}")
print(f"🪣 S3 Bucket Name: {BUCKET_NAME}")

## Step 4: Test Lambda Function

Test the Lambda function directly to ensure it works correctly.

In [ ]:
# Test Lambda function
test_payload = {
    "yml_body": """openapi: 3.0.0
info:
  title: Test API
  version: 1.0.0
paths:
  /users:
    get:
      summary: Get users
    post:
      summary: Create user"""
}

print("🧪 Testing Lambda function...")
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUNCTION_NAME,
    Payload=json.dumps(test_payload)
)

result = json.loads(response['Payload'].read())
print(f"✅ Lambda test result: {result}")

if 'diagramUri' in result:
    print(f"📍 S3 URI: {result['diagramUri']}")
    print(f"📝 PlantUML code length: {len(result.get('codeBody', ''))}")

## Step 5: Create AgentCore Gateway

Set up the AgentCore Gateway with OAuth authentication and Lambda target integration.

parameterize the uml_gateway name in the code repo

In [ ]:
# Import gateway setup function
import sys
sys.path.append("code")
from gateway_setup import setup_gateway

# Create the gateway with custom name
gateway_config = setup_gateway(lambda_arn, GATEWAY_NAME)
print(f"🎯 Gateway configuration: {gateway_config}")

## Step 6: Test Gateway Integration

Test the Gateway to ensure it can successfully call our Lambda function.

In [ ]:
# Import test functions
import sys
sys.path.append("code")
from test_functions import test_gateway_direct

# Test the gateway
test_s3_uri = test_gateway_direct()
print(f"🎯 Test completed. S3 URI: {test_s3_uri}")

## Step 7: Create Strands Agent

Create the Strands agent with Gateway integration for UML generation.

In [ ]:
# Load Strands Agent v2 code from file
with open("code/swagger_agent_v2.py", "r") as f:
    agent_code = f.read()

print("✅ Strands Agent v2 code loaded from code/swagger_agent_v2.py")
print(f"Code length: {len(agent_code)} characters")

# Display first few lines as preview
print("Code preview:")
print("=" * 40)
for i, line in enumerate(agent_code.split("\n")[:10]):
    print(f"{i+1:2d}: {line}")

## Step 8: Test Agent with Direct Integration

Test the agent directly using the Strands SDK before deploying to AgentCore.

In [ ]:
# Import required modules
import json
import sys
sys.path.append("code")
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

# Helper functions
def create_streamable_http_transport(mcp_url: str, access_token: str):
    return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})

def get_full_tools_list(client):
    """Get all tools with pagination support"""
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

def test_agent_v2_direct():
    """Test Agent v2 directly using Strands SDK"""
    # Load gateway configuration
    with open("gateway_config.json", "r") as f:
        config = json.load(f)

    # Get access token
    print("Getting access token...")
    client = GatewayClient(region_name=config["region"])
    access_token = client.get_access_token_for_cognito(config["client_info"])
    print("✓ Access token obtained\n")

    print("🤖 Testing Agent v2 with Gateway integration")
    print("-" * 60)

    # Setup MCP client
    mcp_client = MCPClient(lambda: create_streamable_http_transport(config["gateway_url"], access_token))

    with mcp_client:
        # List available tools
        tools = get_full_tools_list(mcp_client)
        print(f"📋 Available tools: {[tool.tool_name for tool in tools]}")
        print("-" * 60)

        # Create Strands Agent
        model = BedrockModel(
            model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
            streaming=True,
        )

        agent = Agent(model=model, tools=tools)

        # Test 1: Simple query
        print("\n1. Testing simple query...")
        try:
            response = agent("What tools do you have available?")
            print(f"Agent Response: {response.message.get('content', response)}")
        except Exception as e:
            print(f"Error: {e}")

        # Test 2: UML generation
        print("\n2. Testing UML generation...")
        try:
            response = agent("Generate a UML diagram for a simple API with GET /users endpoint and show me the S3 location")
            print(f"Agent Response: {response.message.get('content', response)}")
        except Exception as e:
            print(f"Error: {e}")

# Run the test
test_agent_v2_direct()

## Step 9: Deploy to Bedrock AgentCore

Deploy the agent to Bedrock AgentCore for production use.

In [ ]:
# Configure and deploy to AgentCore
import subprocess
import os

print("🚀 Deploying Agent v2 to Bedrock AgentCore...")

# Configure the agent
try:
    result = subprocess.run([
        "agentcore", "configure", 
        "--name", "swagger_agent_v2",
        "--entrypoint", "swagger_agent_v2.py",
        "--non-interactive"
    ], capture_output=True, text=True, check=True)
    print("✅ Agent configured successfully")
except subprocess.CalledProcessError as e:
    print(f"❌ Configuration failed: {e.stderr}")

# Launch the agent
try:
    result = subprocess.run([
        "agentcore", "launch"
    ], capture_output=True, text=True, check=True)
    print("✅ Agent deployed successfully")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"❌ Deployment failed: {e.stderr}")

## Step 11: Comprehensive Agent Testing with Direct Integration

Test the agent directly using the Strands SDK with the Gateway for comprehensive validation.

In [ ]:
# Import comprehensive test functions
import sys
sys.path.append("code")
from test_functions import test_agent_comprehensive

# Run comprehensive tests with proper error handling
try:
    print("🔐 Getting access token...")
    
    # Load gateway configuration
    with open("gateway_config.json", "r") as f:
        config = json.load(f)
    
    from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
    client = GatewayClient(region_name=config["region"])
    access_token = client.get_access_token_for_cognito(config["client_info"])
    print("✅ Access token obtained")
    
    # Setup MCP client and agent
    from strands.tools.mcp.mcp_client import MCPClient
    from mcp.client.streamable_http import streamablehttp_client
    from strands import Agent
    from strands.models import BedrockModel
    import re
    
    def create_streamable_http_transport(mcp_url: str, access_token: str):
        return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})
    
    def get_full_tools_list(client):
        more_tools = True
        tools = []
        pagination_token = None
        while more_tools:
            tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
            tools.extend(tmp_tools)
            if tmp_tools.pagination_token is None:
                more_tools = False
            else:
                more_tools = True
                pagination_token = tmp_tools.pagination_token
        return tools
    
    print("🔧 Setting up MCP client and agent...")
    mcp_client = MCPClient(lambda: create_streamable_http_transport(config["gateway_url"], access_token))
    
    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        print(f"📋 Available tools: {[tool.tool_name for tool in tools]}")
        
        model = BedrockModel(
            model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
            streaming=True,
        )
        
        agent = Agent(model=model, tools=tools)
        print("✅ Agent created successfully")
        
        # Test 1: Tool availability
        print("\n🧪 Test 1: Tool Availability")
        print("=" * 40)
        try:
            response = agent("What tools do you have available?")
            content = response.message.get("content", [""])[0] if isinstance(response.message.get("content"), list) else str(response.message.get("content", ""))
            print(f"✅ Response: {content[:300]}...")
        except Exception as e:
            print(f"❌ Test 1 failed: {e}")
        
        # Test 2: Simple UML Generation
        print("\n🧪 Test 2: Simple UML Generation")
        print("=" * 40)
        try:
            response = agent("Generate a UML diagram for a simple API with GET /users endpoint and show me the S3 location")
            content = response.message.get("content", [""])[0] if isinstance(response.message.get("content"), list) else str(response.message.get("content", ""))
            print(f"✅ Response: {content[:500]}...")
            
            # Check for S3 URI
            if "s3://" in content:
                print("✅ S3 URI found in response")
                s3_match = re.search(r"s3://[^\s]+", content)
                if s3_match:
                    s3_uri = s3_match.group()
                    print(f"📍 S3 URI: {s3_uri}")
                    with open("test_s3_uris.txt", "a") as f:
                        f.write(f"{s3_uri}\n")
            else:
                print("⚠️ No S3 URI found in response")
        except Exception as e:
            print(f"❌ Test 2 failed: {e}")
        
        print("\n🎯 Comprehensive testing completed!")
        
except Exception as e:
    print(f"❌ Comprehensive testing failed: {e}")

In [ ]:
# Test 4: Custom Query Test
import json
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
import re

def create_streamable_http_transport(mcp_url: str, access_token: str):
    return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})

def get_full_tools_list(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

# Custom query - modify this as needed
custom_query = "Generate a UML diagram for a banking API with account management, transactions, and user authentication. Show me the S3 location."

print(f"🧪 Testing Custom Query: {custom_query[:60]}...")
print("=" * 50)

try:
    with open("gateway_config.json", "r") as f:
        config = json.load(f)
    
    client = GatewayClient(region_name=config["region"])
    access_token = client.get_access_token_for_cognito(config["client_info"])
    
    mcp_client = MCPClient(lambda: create_streamable_http_transport(config["gateway_url"], access_token))
    
    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        model = BedrockModel(model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0", streaming=True)
        agent = Agent(model=model, tools=tools)
        
        response = agent(custom_query)
        content = response.message.get("content", [""])[0] if isinstance(response.message.get("content"), list) else str(response.message.get("content", ""))
        
        print("✅ Response:")
        print(content)
        
        # Extract S3 URI
        if "s3://" in content:
            s3_matches = re.findall(r"s3://[^\s]+", content)
            if s3_matches:
                print(f"\n📍 S3 URIs found: {s3_matches}")
                with open("custom_test_s3_uris.txt", "w") as f:
                    for uri in s3_matches:
                        f.write(f"{uri}\n")
                print("💾 S3 URIs saved to custom_test_s3_uris.txt")
        
except Exception as e:
    print(f"❌ Error: {e}")